In [1]:
"""
Führt Regressionsanalyse durch

"""
import pandas as pd
import numpy as np



#Initmessage
print ("starting Regression") 

#lade von CSV
data_frame_investigated = pd.read_csv("Data/crypto-markets.csv", sep="," )


print("data collected")

print(data_frame_investigated.head())

starting Regression
data collected
      slug symbol     name        date  ranknow    open    high     low  \
0  bitcoin    BTC  Bitcoin  2013-04-28        1  135.30  135.98  132.10   
1  bitcoin    BTC  Bitcoin  2013-04-29        1  134.44  147.49  134.00   
2  bitcoin    BTC  Bitcoin  2013-04-30        1  144.00  146.93  134.05   
3  bitcoin    BTC  Bitcoin  2013-05-01        1  139.00  139.89  107.72   
4  bitcoin    BTC  Bitcoin  2013-05-02        1  116.38  125.60   92.28   

    close  volume        market  close_ratio  spread  
0  134.21     0.0  1.500520e+09       0.5438    3.88  
1  144.54     0.0  1.491160e+09       0.7813   13.49  
2  139.00     0.0  1.597780e+09       0.3843   12.88  
3  116.99     0.0  1.542820e+09       0.2882   32.17  
4  105.21     0.0  1.292190e+09       0.3881   33.32  


In [2]:
#eliminiere redundante Attribute
REDUNDANT_ATTRIBUTES=['symbol','name'] 
data_frame_investigated.drop(REDUNDANT_ATTRIBUTES,axis=1,inplace=True) #loeschen aus Dataframe

#set Date as Index


print(data_frame_investigated.head())


      slug        date  ranknow    open    high     low   close  volume  \
0  bitcoin  2013-04-28        1  135.30  135.98  132.10  134.21     0.0   
1  bitcoin  2013-04-29        1  134.44  147.49  134.00  144.54     0.0   
2  bitcoin  2013-04-30        1  144.00  146.93  134.05  139.00     0.0   
3  bitcoin  2013-05-01        1  139.00  139.89  107.72  116.99     0.0   
4  bitcoin  2013-05-02        1  116.38  125.60   92.28  105.21     0.0   

         market  close_ratio  spread  
0  1.500520e+09       0.5438    3.88  
1  1.491160e+09       0.7813   13.49  
2  1.597780e+09       0.3843   12.88  
3  1.542820e+09       0.2882   32.17  
4  1.292190e+09       0.3881   33.32  


In [3]:

from sklearn.model_selection import train_test_split #library fuer Split
from sklearn import linear_model #lineare Regression
import math #import fuer msqError
from sklearn.metrics import mean_squared_error

#dependency :patsy

def cryptoregress(crypto_id, feature_to_compare,crypto_id_feature_name="slug",
                 documentMode=False):
    """
    lineare Regression zur Bestimmung eines Features anhand gleichen Features der anderen Cryptos
    """
    #Cryptos Gruppieren
    
    #ich kann nur anhand von Tagen schliessen, an denen die Crypto Existiert
    #erstmal alle Tage holen an denen die Crypto gehandelt wird
    dates_of_crypto_trade= data_frame_investigated[
        data_frame_investigated[crypto_id_feature_name] == crypto_id
    ]['date']
    

   
    if documentMode: print(dates_of_crypto_trade.head())
    
    #dann die Tupel Filtern, nur die Tage an denen Crypto getraded wird uebernehmen,
    #nur compare feature uebernehmen
    data_frame_id_exists= data_frame_investigated[
        data_frame_investigated['date'].isin(dates_of_crypto_trade)
    ][['date',crypto_id_feature_name,feature_to_compare]]
    
    
    
    #daten Pivotieren so dass Cryptos zu columns werden
    data_frame_id_exists= data_frame_id_exists.pivot(
        index='date', columns=crypto_id_feature_name, values=feature_to_compare
        ).reset_index()
    
    
    #collumns loeschen die Nullwerte enthalten
    data_frame_id_exists=data_frame_id_exists.dropna(axis='columns')
 
    
   # data_frame_id_exists=data_frame_id_exists.fillna(value=0)
    
    
   
    #date rausloeschen
    data_frame_id_exists.drop('date',axis=1,inplace=True)
    if documentMode: print("cleaned Frame")
    if documentMode: print(data_frame_id_exists.head())
    
    ###############Train Test Split:
    #Ergebnis vom Features trennen:
    Y = data_frame_id_exists[[crypto_id]]
    data_frame_id_exists.drop(crypto_id,axis=1, inplace = True)
    if documentMode: print("Regress to:")
    if documentMode: print(Y.head())

    
        
    feature_train, feature_test, y_train, y_test = train_test_split (
        data_frame_id_exists,
        Y,
        test_size=0.1
    )

    ##############Regressionsmodell machen
    regression_model = linear_model.LinearRegression(
        fit_intercept=True, normalize=False, copy_X=True, n_jobs=-1)


        #regressionsmodell fitten
    regression_model.fit(feature_train, y_train)
    Rsquared_train = regression_model.score(feature_train,y_train)
    Rsqared_test = regression_model.score(feature_test,y_test)
    MeanError_train = math.sqrt( mean_squared_error(regression_model.predict(feature_test),y_test))
    MeanError_test = math.sqrt( mean_squared_error(regression_model.predict(feature_train),y_train))
                                    
    if documentMode: print("RSquared train:",Rsquared_train)
    if documentMode: print("RSquared test:",Rsqared_test)
    if documentMode: print("MeanSqure train:", MeanError_train)
    if documentMode: print("MeanSqure test:", MeanError_test)
    
    return [regression_model,
            Rsquared_train,
            Rsqared_test,
            MeanError_train,
            MeanError_test,
            list(data_frame_id_exists),
           ]
       

    


regression_result=cryptoregress('bitcoin','close')

    
    
    
    
    

In [4]:
print(regression_result)

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False), 0.9323972551645056, 0.9372010914492824, 658.6284712639068, 889.5145139725108, ['litecoin', 'namecoin']]


In [5]:
#make a  list of the different coins
different_coins = data_frame_investigated.slug.unique()
print(different_coins)

['bitcoin' 'ethereum' 'ripple' ... 'sisa' 'entcash' 'numuscash']


In [6]:
#iteriere durch die Liste und fuehre Regressionsanalyse durch
list_regression = []
zaehler=0
if False:
    for current_coin_to_regress in different_coins:
        list_regression.append(
            {
                'name':current_coin_to_regress,
                'RsquaredTest': cryptoregress(current_coin_to_regress,'close')[2]
            }) #fuege ergebnis zur Ergebnistabelle hinzu
        zaehler=zaehler+1
        print("regressed Number ", zaehler, " von ", len(different_coins))
        if zaehler == 1300: break

    zaehler= 0
    df_regression_result=pd.DataFrame(list_regression)
    print(df_regression_result)

In [7]:
#df_regression_result.to_csv("regression_results.csv",sep=',') #Regrssionsergebnisse in csv Speichern

In [8]:
######Importiere Plotly Tools
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go 

"""
Graphische Darstellung von der Verteilung von Rsquared als Histogramm
"""


df_regression_result = pd.read_csv("regression_results.csv",sep=',') #Regressionsergebnisse wieder reinladen


#Histogrammobjekt erstellen
trace0 = go.Histogram(
    x= df_regression_result['RsquaredTest'],histnorm='probability',
    autobinx=False,
    xbins=dict(
        start=0,
        end=1,
        size=0.01
    ),
    cumulative=dict(enabled=True)
    
           

)

data = [trace0]
layout = go.Layout(
    title='RSquared Verteilung',
    xaxis = dict(range=[0, 1], title = 'Rsquared Werte der Coin Regressionen'),
    xaxis = dict(range=[0, 1], title = 'kumulierte Verteilung')

)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cryto-regression')

SyntaxError: keyword argument repeated (<ipython-input-8-b8cbf47d4d76>, line 34)